# Data Cleaning

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:

# Load your CSV Files

customers = pd.read_csv("Data/customers.csv")
inventory = pd.read_csv("Data/inventory.csv")
products = pd.read_csv("Data/products.csv")
orders = pd.read_csv("Data/orders.csv")
order_items = pd.read_csv("Data/order_items.csv")


# Table1 Customers


In [4]:
customers.head()

,customer_id,name,gender,age,country,city,signup_date
0,1,Customer_1,Male,59.0,Jordan,Cairo,2/2/2023
1,2,Customer_2,Female,38.0,Lebanon,Amman,11/4/2023
2,3,Customer_3,Male,54.0,Jordan,NaN,6/14/2023
3,4,Customer_4,Male,44.0,UAE,Riyadh,11/25/2023
4,5,Customer_5,Male,46.0,Saudi Arabia,Riyadh,4/29/2023


In [5]:
customers.shape

(8240, 7)

In [6]:
customers.columns

Index(['customer_id', 'name', 'gender', 'age', 'country', 'city',
       'signup_date'],
      dtype='str')

In [7]:
customers.info()

<class 'pandas.DataFrame'>
RangeIndex: 8240 entries, 0 to 8239
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   customer_id  8240 non-null   int64  
 1   name         8240 non-null   str    
 2   gender       8240 non-null   str    
 3   age          7591 non-null   float64
 4   country      8240 non-null   str    
 5   city         7585 non-null   str    
 6   signup_date  8240 non-null   str    
dtypes: float64(1), int64(1), str(5)
memory usage: 450.8 KB


In [8]:
customers.isna().sum()

customer_id      0
name             0
gender           0
age            649
country          0
city           655
signup_date      0
dtype: int64

In [9]:
customers.isnull().sum()

customer_id      0
name             0
gender           0
age            649
country          0
city           655
signup_date      0
dtype: int64

In [10]:
customers.describe()

,customer_id,age
count,8240.000000,7591.000000
mean,4007.447937,40.957186
std,2312.895129,13.411728
min,1.000000,18.000000
25%,2000.750000,30.000000
50%,4007.500000,41.000000
75%,6009.250000,52.500000
max,8000.000000,64.000000


# Handle Missing Values

Fix Age

🔹 Step 1: Create the flag BEFORE imputing

In [11]:
# I created an imputation flag before filling missing values 
# so downstream analysis can distinguish between original and imputed data.
# This preserves transparency and prevents hidden bias.

customers['age_imputed'] = customers['age'].isna().astype(int)

🔹 Step 2: Impute missing age with median

In [12]:
# I used median imputation for age because age distributions are typically skewed and may contain outliers. 
# Median is more robust than mean and better represents the central tendency of customer age.

customers['age'] = customers['age'].fillna(customers['age'].median())

Validation

In [13]:
# ✔ That confirms the flag is correct
# ✔ No information lost
# ✔ Analysis stays honest
customers['age_imputed'].value_counts()

age_imputed
0    7591
1     649
Name: count, dtype: int64

Fix City

In [14]:
customers['city'] = customers['city'].fillna("Unknown")

In [15]:
customers.isnull().sum()

customer_id    0
name           0
gender         0
age            0
country        0
city           0
signup_date    0
age_imputed    0
dtype: int64

In [16]:
customers.head()

,customer_id,name,gender,age,country,city,signup_date,age_imputed
0,1,Customer_1,Male,59.0,Jordan,Cairo,2/2/2023,0
1,2,Customer_2,Female,38.0,Lebanon,Amman,11/4/2023,0
2,3,Customer_3,Male,54.0,Jordan,Unknown,6/14/2023,0
3,4,Customer_4,Male,44.0,UAE,Riyadh,11/25/2023,0
4,5,Customer_5,Male,46.0,Saudi Arabia,Riyadh,4/29/2023,0


In [17]:
customers.groupby('age_imputed')['customer_id'].count()

age_imputed
0    7591
1     649
Name: customer_id, dtype: int64

# Remove Duplicates

In [18]:
customers.duplicated().sum()


np.int64(240)

In [19]:
customers[customers.duplicated()]


,customer_id,name,gender,age,country,city,signup_date,age_imputed
8000,457,Customer_457,Female,21.0,Jordan,Cairo,6/11/2024,0
8001,1722,Customer_1722,Male,44.0,Saudi Arabia,Riyadh,4/23/2023,0
8002,4417,Customer_4417,Male,22.0,UAE,Amman,6/20/2024,0
8003,3672,Customer_3672,Female,24.0,UAE,Beirut,5/6/2023,0
8004,3566,Customer_3566,Female,49.0,UAE,Dubai,5/2/2024,0
...,...,...,...,...,...,...,...,...
8235,5652,Customer_5652,Male,34.0,Saudi Arabia,Riyadh,4/4/2023,0
8236,4350,Customer_4350,Male,30.0,Saudi Arabia,Amman,12/11/2023,0
8237,7580,Customer_7580,Female,37.0,Egypt,Amman,1/15/2023,0
8238,2396,Customer_2396,Female,26.0,Lebanon,Riyadh,8/6/2023,0


In [20]:
customers['customer_id'].duplicated().sum()


np.int64(240)

Remove Duplicates by Primary Key

In [21]:
customers = customers.drop_duplicates(subset='customer_id', keep='first')


In [22]:
customers.duplicated().sum()
customers['customer_id'].duplicated().sum()


np.int64(0)

In [23]:
customers.shape


(8000, 8)

The customers table contained 240 duplicate customer_id values, violating primary key uniqueness. 

Since duplicate records contained identical attributes, duplicates were resolved by retaining the first occurrence per customer_id

# Correct Data Types

In [24]:
customers['signup_date'] = pd.to_datetime(customers['signup_date'], errors='coerce')


In [25]:
customers['age'] = customers['age'].astype(int)


In [26]:
customers.info()


<class 'pandas.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  8000 non-null   int64         
 1   name         8000 non-null   str           
 2   gender       8000 non-null   str           
 3   age          8000 non-null   int64         
 4   country      8000 non-null   str           
 5   city         8000 non-null   str           
 6   signup_date  8000 non-null   datetime64[us]
 7   age_imputed  8000 non-null   int64         
dtypes: datetime64[us](1), int64(3), str(4)
memory usage: 500.1 KB


In [27]:
customers['gender'].unique()
customers['country'].unique()
customers['city'].unique()


<StringArray>
['Cairo', 'Amman', 'Unknown', 'Riyadh', 'Dubai', 'Beirut']
Length: 6, dtype: str

# Identify Outliers (Customers Table)

In [28]:
customers.describe()

,customer_id,age,signup_date,age_imputed
count,8000.00000,8000.000000,8000,8000.00000
mean,4000.50000,40.936500,2023-10-26 21:32:45.600000,0.08000
min,1.00000,18.000000,2023-01-01 00:00:00,0.00000
25%,2000.75000,30.000000,2023-05-29 18:00:00,0.00000
50%,4000.50000,41.000000,2023-10-26 00:00:00,0.00000
75%,6000.25000,51.000000,2024-03-24 00:00:00,0.00000
max,8000.00000,64.000000,2024-08-23 00:00:00,1.00000
std,2309.54541,12.873692,NaN,0.27131


Age distribution was assessed using descriptive statistics. Values ranged from 18 to 64, with no extreme outliers detected. Therefore, no outlier treatment was required.

# Final Quality Check

In [29]:
customers.isna().sum()
customers.duplicated().sum()
customers['customer_id'].duplicated().sum()
customers.info()


<class 'pandas.DataFrame'>
RangeIndex: 8000 entries, 0 to 7999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   customer_id  8000 non-null   int64         
 1   name         8000 non-null   str           
 2   gender       8000 non-null   str           
 3   age          8000 non-null   int64         
 4   country      8000 non-null   str           
 5   city         8000 non-null   str           
 6   signup_date  8000 non-null   datetime64[us]
 7   age_imputed  8000 non-null   int64         
dtypes: datetime64[us](1), int64(3), str(4)
memory usage: 500.1 KB


# Table 2 Inventory

The inventory table tracks how many units of each product are stored in each warehouse, when to reorder them, and when the data was last updated.

In [30]:
inventory.head()

,product_id,warehouse,stock_level,reorder_point,last_updated
0,1,WH_A,54,117,1/19/2023
1,2,WH_A,242,76,1/13/2023
2,3,WH_A,31,93,1/26/2023
3,4,WH_C,153,65,1/4/2023
4,5,WH_C,23,94,1/25/2023


In [31]:
inventory.info()

<class 'pandas.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype
---  ------         --------------  -----
 0   product_id     1200 non-null   int64
 1   warehouse      1200 non-null   str  
 2   stock_level    1200 non-null   int64
 3   reorder_point  1200 non-null   int64
 4   last_updated   1200 non-null   str  
dtypes: int64(3), str(2)
memory usage: 47.0 KB


In [32]:
inventory.shape

(1200, 5)

In [33]:
inventory.columns

Index(['product_id', 'warehouse', 'stock_level', 'reorder_point',
       'last_updated'],
      dtype='str')

# Correct Data Types

In [34]:
inventory['last_updated'] = pd.to_datetime(inventory['last_updated'], errors='coerce')

In [35]:
inventory.info()

<class 'pandas.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   product_id     1200 non-null   int64         
 1   warehouse      1200 non-null   str           
 2   stock_level    1200 non-null   int64         
 3   reorder_point  1200 non-null   int64         
 4   last_updated   1200 non-null   datetime64[us]
dtypes: datetime64[us](1), int64(3), str(1)
memory usage: 47.0 KB


In [36]:
inventory.describe()

,product_id,stock_level,reorder_point,last_updated
count,1200.000000,1200.000000,1200.000000,1200
mean,600.500000,261.438333,98.430000,2023-01-15 21:02:24
min,1.000000,-38.000000,50.000000,2023-01-01 00:00:00
25%,300.750000,131.000000,72.000000,2023-01-08 00:00:00
50%,600.500000,257.000000,98.000000,2023-01-16 00:00:00
75%,900.250000,382.250000,124.000000,2023-01-24 00:00:00
max,1200.000000,788.000000,149.000000,2023-01-31 00:00:00
std,346.554469,159.263027,29.160625,NaN


# Missing Values

In [37]:
inventory.isna().sum()

product_id       0
warehouse        0
stock_level      0
reorder_point    0
last_updated     0
dtype: int64

# Handle Duplicates

In [38]:
inventory.duplicated().sum()

np.int64(0)

# Checking Outliers

In [39]:
inventory.describe()

,product_id,stock_level,reorder_point,last_updated
count,1200.000000,1200.000000,1200.000000,1200
mean,600.500000,261.438333,98.430000,2023-01-15 21:02:24
min,1.000000,-38.000000,50.000000,2023-01-01 00:00:00
25%,300.750000,131.000000,72.000000,2023-01-08 00:00:00
50%,600.500000,257.000000,98.000000,2023-01-16 00:00:00
75%,900.250000,382.250000,124.000000,2023-01-24 00:00:00
max,1200.000000,788.000000,149.000000,2023-01-31 00:00:00
std,346.554469,159.263027,29.160625,NaN


In [40]:
(inventory['stock_level'] < 0).sum()


np.int64(6)

In [41]:
inventory[inventory['stock_level'] < 0]


,product_id,warehouse,stock_level,reorder_point,last_updated
58,59,WH_A,-13,125,2023-01-01
61,62,WH_C,-19,68,2023-01-29
180,181,WH_A,-38,83,2023-01-12
695,696,WH_C,-35,77,2023-01-31
774,775,WH_B,-11,95,2023-01-30
903,904,WH_C,-4,131,2023-01-21


In [42]:
inventory = inventory[inventory['stock_level'] >= 0]


In [43]:
(inventory['stock_level'] < 0).sum()

np.int64(0)

In [44]:
inventory.describe()

,product_id,stock_level,reorder_point,last_updated
count,1194.000000,1194.000000,1194.000000,1194
mean,601.275544,262.852596,98.439698,2023-01-15 20:27:44.321608
min,1.000000,0.000000,50.000000,2023-01-01 00:00:00
25%,302.250000,135.250000,72.000000,2023-01-08 00:00:00
50%,600.500000,258.500000,98.000000,2023-01-16 00:00:00
75%,900.750000,383.000000,123.750000,2023-01-24 00:00:00
max,1200.000000,788.000000,149.000000,2023-01-31 00:00:00
std,346.347623,158.401804,29.185049,NaN


Inventory data contained 6 records with negative stock levels, which violate business rules. 
These records were identified, reviewed, and removed. Post-cleaning validation confirmed all stock levels are non-negative.

# Table 3 Order_items

The orders table stores transaction-level information, while order_items contains product-level details for each order. They are linked by order_id in a one-to-many relationship.



In [45]:
order_items.head()

,order_item_id,order_id,product_id,quantity,unit_price
0,1,1,710,2,271.79
1,2,1,780,3,296.09
2,3,2,414,2,332.99
3,4,2,236,5,30.91
4,5,3,147,5,13.68


In [46]:
order_items.shape

(149834, 5)

In [47]:
order_items.columns

Index(['order_item_id', 'order_id', 'product_id', 'quantity', 'unit_price'], dtype='str')

In [48]:
order_items.info()

<class 'pandas.DataFrame'>
RangeIndex: 149834 entries, 0 to 149833
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   order_item_id  149834 non-null  int64  
 1   order_id       149834 non-null  int64  
 2   product_id     149834 non-null  int64  
 3   quantity       149834 non-null  int64  
 4   unit_price     149834 non-null  float64
dtypes: float64(1), int64(4)
memory usage: 5.7 MB


In [49]:
order_items.duplicated().sum()

np.int64(0)

# Checking outliers

In [50]:
order_items.describe()

,order_item_id,order_id,product_id,quantity,unit_price
count,149834.000000,149834.000000,149834.000000,149834.000000,149834.000000
mean,74917.500000,29971.535299,599.474512,25.294993,274.790241
std,43253.494454,17304.499486,346.863209,120.204276,171.983929
min,1.000000,1.000000,1.000000,-100.000000,7.350000
25%,37459.250000,15004.000000,298.000000,2.000000,134.480000
50%,74917.500000,29921.000000,599.000000,3.000000,252.690000
75%,112375.750000,44947.000000,900.000000,4.000000,399.830000
max,149834.000000,60000.000000,1200.000000,999.000000,739.840000


In [51]:
(order_items['quantity'] <= 0).sum()


np.int64(693)

In [52]:
order_items[order_items['quantity'] <= 0]


,order_item_id,order_id,product_id,quantity,unit_price
33,34,16,48,-82,350.25
411,412,166,1107,-26,420.42
532,533,216,1193,-63,56.64
653,654,269,728,-57,555.47
934,935,374,320,-4,320.57
...,...,...,...,...,...
149255,149256,59770,365,-60,30.83
149393,149394,59827,1147,-85,88.70
149406,149407,59833,330,-92,12.44
149719,149720,59955,44,-25,452.47


Remove Invalid Quantities

Records with non-positive quantities were removed

In [53]:
order_items = order_items[order_items['quantity'] > 0]


In [54]:
(order_items['quantity'] <= 0).sum()


np.int64(0)

In [55]:
order_items.describe()

,order_item_id,order_id,product_id,quantity,unit_price
count,149141.000000,149141.000000,149141.000000,149141.000000,149141.000000
mean,74921.875319,29973.282960,599.594404,25.640287,274.754093
std,43250.838019,17303.439322,346.804642,120.358864,171.962191
min,1.000000,1.000000,1.000000,1.000000,7.350000
25%,37473.000000,15010.000000,298.000000,2.000000,134.480000
50%,74911.000000,29919.000000,599.000000,3.000000,252.690000
75%,112383.000000,44949.000000,900.000000,4.000000,399.060000
max,149834.000000,60000.000000,1200.000000,999.000000,739.840000


# Table 4 Orders

In [56]:
orders.head()

,order_id,customer_id,order_date,status,payment_method,region
0,1,4506,1/4/2023,Cancelled,Credit Card,Gulf
1,2,2881,7/18/2023,Completed,Cash,Levant
2,3,7248,12/17/2023,Completed,Cash,North Africa
3,4,5808,1/8/2024,Completed,PayPal,Gulf
4,5,6257,5/9/2023,Completed,Credit Card,North Africa


In [57]:
orders.shape

(62400, 6)

In [58]:
orders.columns

Index(['order_id', 'customer_id', 'order_date', 'status', 'payment_method',
       'region'],
      dtype='str')

In [59]:
orders.info()

<class 'pandas.DataFrame'>
RangeIndex: 62400 entries, 0 to 62399
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   order_id        62400 non-null  int64
 1   customer_id     62400 non-null  int64
 2   order_date      62400 non-null  str  
 3   status          62400 non-null  str  
 4   payment_method  58660 non-null  str  
 5   region          62400 non-null  str  
dtypes: int64(2), str(4)
memory usage: 2.9 MB


# Correct Data Types

In [60]:
orders['order_date'] = pd.to_datetime(orders['order_date'], errors='coerce')

In [61]:
orders.info()

<class 'pandas.DataFrame'>
RangeIndex: 62400 entries, 0 to 62399
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        62400 non-null  int64         
 1   customer_id     62400 non-null  int64         
 2   order_date      62400 non-null  datetime64[us]
 3   status          62400 non-null  str           
 4   payment_method  58660 non-null  str           
 5   region          62400 non-null  str           
dtypes: datetime64[us](1), int64(2), str(3)
memory usage: 2.9 MB


# checking Duplicates

In [62]:
orders.duplicated().sum()

np.int64(2400)

In [64]:
orders['order_id'].duplicated().sum()

np.int64(2400)

In [65]:
orders[orders['order_id'].duplicated(keep=False)].sort_values('order_id')


,order_id,customer_id,order_date,status,payment_method,region
61788,7,6965,2024-08-26,Completed,Credit Card,North Africa
6,7,6965,2024-08-26,Completed,Credit Card,North Africa
60644,24,722,2024-11-04,Completed,Bank Transfer,Gulf
23,24,722,2024-11-04,Completed,Bank Transfer,Gulf
28,29,5804,2023-03-01,Completed,Credit Card,Levant
...,...,...,...,...,...,...
59885,59886,3720,2023-07-14,Completed,PayPal,Gulf
59926,59927,2328,2023-03-07,Completed,PayPal,North Africa
60007,59927,2328,2023-03-07,Completed,PayPal,North Africa
59984,59985,1336,2023-09-30,Cancelled,Cash,North Africa


# Remove Duplicates

In [66]:
orders = orders.drop_duplicates(subset='order_id', keep='first')

In [69]:
orders['order_id'].duplicated().sum()
orders.duplicated().sum()

np.int64(0)

# Handle Missing Values

In [70]:
orders.info()

<class 'pandas.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        60000 non-null  int64         
 1   customer_id     60000 non-null  int64         
 2   order_date      60000 non-null  datetime64[us]
 3   status          60000 non-null  str           
 4   payment_method  56400 non-null  str           
 5   region          60000 non-null  str           
dtypes: datetime64[us](1), int64(2), str(3)
memory usage: 2.7 MB


In [72]:
orders['payment_method'] = orders['payment_method'].fillna('Unknown')

In [73]:
orders.info()

<class 'pandas.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   order_id        60000 non-null  int64         
 1   customer_id     60000 non-null  int64         
 2   order_date      60000 non-null  datetime64[us]
 3   status          60000 non-null  str           
 4   payment_method  60000 non-null  str           
 5   region          60000 non-null  str           
dtypes: datetime64[us](1), int64(2), str(3)
memory usage: 2.7 MB


In [74]:
orders.isna().sum()

order_id          0
customer_id       0
order_date        0
status            0
payment_method    0
region            0
dtype: int64

The orders table contained missing values in the payment_method field (~6%). 
Since payment type cannot be reliably inferred, missing values were labeled as ‘Unknown’ to preserve analytical integrity.

In [75]:
orders.describe()

,order_id,customer_id,order_date
count,60000.000000,60000.000000,60000
mean,30000.500000,4005.832850,2023-12-16 19:27:14.399999
min,1.000000,1.000000,2023-01-01 00:00:00
25%,15000.750000,2016.000000,2023-06-24 00:00:00
50%,30000.500000,4017.000000,2023-12-16 00:00:00
75%,45000.250000,6008.000000,2024-06-08 00:00:00
max,60000.000000,8000.000000,2024-12-01 00:00:00
std,17320.652413,2306.375401,NaN


# Final Quality check

In [ ]:
orders.isna().sum()
orders.duplicated().sum()
orders['order_id'].duplicated().sum()
orders.info()

order_id          0
customer_id       0
order_date        0
status            0
payment_method    0
region            0
dtype: int64

# Table 5 Products

In [84]:
products.head()

,product_id,product_name,category,sub_category,cost,price
0,1,Product_1,Home,Kitchen,244.11,588.96
1,2,Product_2,Sports,Fitness,91.88,138.91
2,3,Product_3,Fashion,Women,111.04,168.73
3,4,Product_4,Fashion,Kids,294.49,712.85
4,5,Product_5,Sports,Outdoor,63.80,136.93


In [85]:
products.shape

(1200, 6)

In [86]:
products.columns

Index(['product_id', 'product_name', 'category', 'sub_category', 'cost',
       'price'],
      dtype='str')

In [87]:
products.info()

<class 'pandas.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    1200 non-null   int64  
 1   product_name  1200 non-null   str    
 2   category      1200 non-null   str    
 3   sub_category  1140 non-null   str    
 4   cost          1200 non-null   float64
 5   price         1200 non-null   float64
dtypes: float64(2), int64(1), str(3)
memory usage: 56.4 KB


In [89]:
products.isna().sum()

product_id       0
product_name     0
category         0
sub_category    60
cost             0
price            0
dtype: int64

In [88]:
products.duplicated().sum()

np.int64(0)

# Handle Missing Values

In [90]:
products['sub_category'] = products['sub_category'].fillna('Unknown')

In [91]:
products.isna().sum()

product_id      0
product_name    0
category        0
sub_category    0
cost            0
price           0
dtype: int64

# Check Outliers

In [93]:
products.describe()

,product_id,cost,price
count,1200.000000,1200.000000,1200.000000
mean,600.500000,150.353992,283.857608
std,346.554469,86.273167,184.026353
min,1.000000,5.760000,-46.000000
25%,300.750000,77.865000,136.417500
50%,600.500000,146.580000,259.245000
75%,900.250000,226.295000,410.192500
max,1200.000000,299.870000,977.000000


In [94]:
(products['price'] < 0).sum()

np.int64(2)

In [95]:
products[products['price'] < 0]


,product_id,product_name,category,sub_category,cost,price
87,88,Product_88,Fashion,Unknown,52.97,-46.0
524,525,Product_525,Fashion,Men,94.14,-28.0


In [96]:
products = products[products['price'] >= 0]

In [97]:
(products['price'] < 0).sum()

np.int64(0)

In [98]:
products.describe()

,product_id,cost,price
count,1198.000000,1198.000000,1198.000000
mean,600.990818,150.482204,284.393264
std,346.520182,86.283928,183.711316
min,1.000000,5.760000,7.350000
25%,301.250000,77.932500,136.825000
50%,601.500000,146.620000,259.835000
75%,900.750000,226.365000,410.277500
max,1200.000000,299.870000,977.000000


# Final Quality Check

In [106]:
products.isna().sum()
products.duplicated().sum()
products['product_id'].duplicated().sum()
products.info()

<class 'pandas.DataFrame'>
Index: 1198 entries, 0 to 1199
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    1198 non-null   int64  
 1   product_name  1198 non-null   str    
 2   category      1198 non-null   str    
 3   sub_category  1198 non-null   str    
 4   cost          1198 non-null   float64
 5   price         1198 non-null   float64
dtypes: float64(2), int64(1), str(3)
memory usage: 65.5 KB


In [100]:
products[products['price'] < products['cost']]

,product_id,product_name,category,sub_category,cost,price
88,89,Product_89,Sports,Clothing,259.57,50.0
384,385,Product_385,Fashion,Women,239.57,109.0
396,397,Product_397,Electronics,Laptops,55.09,53.0
496,497,Product_497,Electronics,Phones,109.03,99.0
613,614,Product_614,Sports,Fitness,169.08,92.0


In [107]:
products = products[products['price'] >= products['cost']]


In [108]:
(products['price'] < products['cost']).sum()

np.int64(0)

Cost represents the per-unit procurement cost paid by the company, while price represents the per-unit selling price charged to customers. 
Since the dataset lacked promotion or discount indicators, records where price was lower than cost were treated as data inconsistencies and removed.

In [109]:
customers.to_csv("Data/customers_cleaned.csv", index=False)
orders.to_csv("Data/orders_cleaned.csv", index=False)
order_items.to_csv("Data/order_items_cleaned.csv", index=False)
products.to_csv("Data/products_cleaned.csv", index=False)
inventory.to_csv("Data/inventory_cleaned.csv", index=False)

In [119]:
orders.head()

,order_id,customer_id,order_date,status,payment_method,region
0,1,4506,2023-01-04,Cancelled,Credit Card,Gulf
1,2,2881,2023-07-18,Completed,Cash,Levant
2,3,7248,2023-12-17,Completed,Cash,North Africa
3,4,5808,2024-01-08,Completed,PayPal,Gulf
4,5,6257,2023-05-09,Completed,Credit Card,North Africa


In [120]:
order_items.head()

,order_item_id,order_id,product_id,quantity,unit_price
0,1,1,710,2,271.79
1,2,1,780,3,296.09
2,3,2,414,2,332.99
3,4,2,236,5,30.91
4,5,3,147,5,13.68


In [121]:
products.head()

,product_id,product_name,category,sub_category,cost,price
0,1,Product_1,Home,Kitchen,244.11,588.96
1,2,Product_2,Sports,Fitness,91.88,138.91
2,3,Product_3,Fashion,Women,111.04,168.73
3,4,Product_4,Fashion,Kids,294.49,712.85
4,5,Product_5,Sports,Outdoor,63.80,136.93
